In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, confusion_matrix, f1_score

In [3]:
import os, sys
sys.path.insert(0, '..')
from src.models.config import SEED
from src.utils import performance_rank_df, performance_rank_n, performance_rank_f1_opt
from src.utils import plot_rank_precision_recall, plot_precision_recall

In [4]:
df = pd.read_csv('../data/raw/creditcard.csv') # pd.read_csv('https://storage.googleapis.com/download.tensorflow.org/data/creditcard.csv')
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


# Data split

In [5]:
y=df['Class']
X=df.drop('Class',1)

X['Amount_log'] = np.log(X['Amount'] + 1)
X=X.drop(['Amount','Time'],1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [6]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
#X_val = sc.transform(X_val)
X_test = sc.transform(X_test)

# Model

## Model hyperparameter optimization

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn import metrics

In [8]:
%%time
# define models and parameters
model = LogisticRegression(random_state=42)
solvers = ['newton-cg', 'lbfgs', 'liblinear']
penalty = ['l2']
c_values = [100, 10, 1.0, 0.1]
# define grid search
grid = dict(solver=solvers,penalty=penalty,C=c_values)
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
search = RandomizedSearchCV(estimator=model, param_distributions=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
search_result = search.fit(X_train, y_train)
# summarize results
print("Best: %f using %s" % (search_result.best_score_, search_result.best_params_))

Best: 0.999220 using {'solver': 'newton-cg', 'penalty': 'l2', 'C': 100}
CPU times: user 7.6 s, sys: 1.23 s, total: 8.82 s
Wall time: 2min 5s


In [9]:
means = search_result.cv_results_['mean_test_score']
stds = search_result.cv_results_['std_test_score']
params = search_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

0.999204 (0.000093) with: {'solver': 'lbfgs', 'penalty': 'l2', 'C': 0.1}
0.999204 (0.000093) with: {'solver': 'newton-cg', 'penalty': 'l2', 'C': 0.1}
0.999220 (0.000099) with: {'solver': 'newton-cg', 'penalty': 'l2', 'C': 100}
0.999213 (0.000097) with: {'solver': 'liblinear', 'penalty': 'l2', 'C': 1.0}
0.999220 (0.000099) with: {'solver': 'liblinear', 'penalty': 'l2', 'C': 10}
0.999220 (0.000099) with: {'solver': 'liblinear', 'penalty': 'l2', 'C': 100}
0.999220 (0.000099) with: {'solver': 'lbfgs', 'penalty': 'l2', 'C': 100}
0.999187 (0.000095) with: {'solver': 'liblinear', 'penalty': 'l2', 'C': 0.1}
0.999220 (0.000099) with: {'solver': 'lbfgs', 'penalty': 'l2', 'C': 10}
0.999216 (0.000101) with: {'solver': 'lbfgs', 'penalty': 'l2', 'C': 1.0}


# Final Model

In [10]:
search_result.best_params_

{'solver': 'newton-cg', 'penalty': 'l2', 'C': 100}

In [15]:
clf = LogisticRegression(random_state=42, solver='newton-cg',penalty='l2',C=100).fit(X_train, y_train)
y_score_test = clf.predict_proba(X_test)[:,1]

In [16]:
fpr, tpr, thresholds = metrics.roc_curve(np.array(y_test), y_score_test, pos_label=1)
metrics.auc(fpr, tpr)

0.9763398958345296

In [17]:
df_pf_test = performance_rank_df(y_test, y_score_test, if_score = False)

In [18]:
performance_rank_n(df_pf_test)

,Precision,Recall,F1_score
Rank,,,
100,0.7900,0.806122,0.797980
500,0.1760,0.897959,0.294314
1000,0.0900,0.918367,0.163934
10000,0.0094,0.959184,0.018618
